# Hierarchical Bayesian model fitting via iterative sampling

This algorithm iteratively computes model evidence via a sampling procedure. Each iteration of the loop a posterior sample for each subject is derived by weighting the distribution of parameters by their likelihood. These samples at the subject level are concatenated, and then hyperparameters are fit to these distributions. Model evidence at the group level is a sum of subject-level evidence (given that is in log space). Once model evidence (defined by iBIC) does not improve, the iterations terminate. 

(1) Sample from a prior distribution over $\forall i$ paramters $\theta_{i}$

(2) Generate likelihoods $p(data|\theta_i)$

(3) Compute posterior distribution over parameters by $p(\theta)\cdot p(data|\theta)$

(4) Fit new hyperparameters to the posterior sample then go back to (1) unless iBIC hasn't improved

## Model class & requisite functions

In [16]:
#function to sample parameters within a model
def sample_parameters(distribution_type,hyperparameter_list,sample_size):
    from numpy.random import beta,gamma,chisquare,normal,poisson,uniform,logistic,multinomial,binomial
    counter=1
    for num in hyperparameter_list:
        exec("global hp_{}; hp_{}={}".format(counter,counter,num))
        counter+=1
    exec("global sample; sample={}({},{},{})".format(distribution_type,hp_1,hp_2,sample_size))
    return sample
    
# MODEL CLASS: assumes a hierarchical model 
# population parameters and subject level parameters 
# are jointly fit

#  The structure of the model class:
#         GROUP LEVEL:
#              Name - E.g., Standard Q Learning
#              Sample Size - How many samples for each parameter
#              Lik - Likelihood function
#              iBIC - Total evidence accounting for model complexity
#              Total_Evidence: Sum of Subject-Level Evidences (sum b/c in log-space)
#              Parameters (entries below x #parameters):
#                     Hyperparameters - e.g., [mu,sigma]
#                     Distribution Type - e.g., Beta
#                     Name - e.g., Lrate-Value
#
#        SUBJECT LEVEL: 
#           Evidence (i.e., log mean likelihood)
#           Parameters (entries below x #parameters):       
#                 posterior mean
#                 credible interval (95%)
#                 samples
#                 num_good_samples (not non or inf samples)


class model:
    
    def __init__(self):
        self.name=0
        self.num_subjects=0
        self.params=self.groupParams()
        self.subjfit=self.subjFit()
        self.subj_level_info=self.subjFit.subjParams()
        self.sample_size=0
        self.model_evidence_group=0
        self.bic=10**10 #arbitrarily high starting iBIC
        self.lik_func=0 #likelihood function
        
    
    class groupParams:
        
        def __init__(self):
            self.name='eta'
            self.distribution='beta'
            self.hyperparameters=[1,2]
                            
    class subjFit:
        def __init__(self):
            self.model_evidence_subject=0
            self.subj_level_info=self.subjParams()
            
        class subjParams:
            def __init__(self):
                self.posterior_mean=0
                self.credible_interval=0
                self.samples=[]
                self.num_good_samples=[] #not nan or inf
            
        

def moodyRL(model,data):
    import numpy as np
    import scipy
    sample_size=model.sample_size
    
    #initialize variables at zero
    q_values=np.zeros((model.sample_size,2)) #initialize values for each sample, assumes 2 choices
    lik=np.zeros((model.sample_size,1))
    mood=np.zeros((model.sample_size,1))
    
    #retrieve samples of parameters from model 
    lr_mood=np.reshape(model.lr_mood.samples,(model.sample_size,1))
    lr_val=np.reshape(model.lr_val.samples,(model.sample_size,1))
    inv_temp=np.reshape(model.inv_temp.samples,(model.sample_size,1))
    mood_bias=np.reshape(model.mood_bias.samples,(model.sample_size,1))
    
    #retrieve relevant data
    choice=data.choice # data = pandas dataframe
    outcome=data.outcome
    
    #calculate log likelihood by iterating over choice data
    for index in range(len(choice)):
        lik = np.add(lik, (np.subtract(np.multiply(inv_temp,np.reshape(q_values[:,choice[index]],(model.sample_size,1))),np.reshape(scipy.special.logsumexp(np.multiply(inv_temp,q_values),axis=1),(model.sample_size,1)))))
        uf=np.tanh(mood); #nonlinear transformation of mood by sigmoid
        perceived_r=np.add(outcome[index],np.multiply(mood_bias,uf))
        pe = np.subtract(perceived_r,np.reshape(q_values[:,choice[index]],(model.sample_size,1)))
        mood=np.add(np.multiply(mood,(1-lr_mood)),np.multiply(lr_mood,pe))
        x = np.add(np.reshape(q_values[:,choice[index]],(model.sample_size,1)), np.multiply(lr_val,pe))   
        q_values[:,choice[index]] = x.flatten()
    return lik

def std_RL(model,data):
    import numpy as np
    import scipy
    sample_size=model.sample_size
    
    #initialize variables at zero
    q_values=np.zeros((model.sample_size,2)) #initialize values for each sample, assumes 2 choices
    lik=np.zeros((model.sample_size,1))
    
    #retrieve samples of parameters from model 
    lr_val=np.reshape(model.lr_val.samples,(model.sample_size,1))
    inv_temp=np.reshape(model.inv_temp.samples,(model.sample_size,1))
    
    #retrieve relevant data
    choice=data.choice # data = pandas dataframe
    outcome=data.outcome
    
    #calculate log likelihood by iterating over choice data
    for index in range(len(choice)):
        lik = np.add(lik, (np.subtract(np.multiply(inv_temp,np.reshape(q_values[:,choice[index]],(model.sample_size,1))),np.reshape(scipy.special.logsumexp(np.multiply(inv_temp,q_values),axis=1),(model.sample_size,1)))))
        pe = np.subtract(outcome[index],np.reshape(q_values[:,choice[index]],(model.sample_size,1)))
        x = np.add(np.reshape(q_values[:,choice[index]],(model.sample_size,1)), np.multiply(lr_val,pe))   
        q_values[:,choice[index]] = x.flatten()
    return lik

#retrieve list of parameters from model
def get_parameters_for_model(model):
    parameters=[]
    for var in vars(model):
        exec('global x; x={}.{}'.format(model.name,var))
        if type(x)==model.groupParams:
            if var!='params':
                parameters.append(var)
    return parameters

def build_model(name,likelihood,group_parameters_info,number_subjects,sample_size):
    #  INPUTS:
    #     name = name of model
    #     likelihood = likelihood function
    #     group_parameter_info = *Python dictionary* 
    #       defining parameter names, distributions and hyperparameters
    #       EXAMPLE: {'eta':['beta',[1,1]]}
    #     sample_size = number of samples from prior over group parameters
    
    #  OUTPUTS:
    #     model class (see above)
    
    exec('{}=model()'.format(name,name),globals())
    exec('{}.name="{}"'.format(name,name))
    exec('{}.num_subjects={}'.format(name,number_subjects))
    exec('{}.lik_func={}'.format(name,likelihood))
    exec('{}.sample_size={}'.format(name,sample_size))
    
    #encode in model the number of subjects and parameters in one's data
    for parameter in group_parameters_info:
        exec('{}.{}={}.groupParams()'.format(name,parameter,name))
        exec('{}.{}.name="{}"'.format(name,parameter,parameter))
        exec('{}.{}.distribution="{}"'.format(name,parameter,group_parameters_info[parameter][0]))
        exec('{}.{}.hyperparameters={}'.format(name,parameter,group_parameters_info[parameter][1]))
        exec('{}.{}.sample_size={}'.format(name,parameter,sample_size))
        exec('{}.{}.samples=sample_parameters("{}",{},{})'.format(name,parameter,group_parameters_info[parameter][0],group_parameters_info[parameter][1],sample_size))

    for sub in range(number_subjects):
        exec('{}.subject{}={}.subjFit()'.format(name,sub,name))
        for parameter in group_parameters_info:
            exec('{}.subject{}.{}={}.subject{}.subjParams()'.format(name,sub,parameter,name,sub))

def compute_new_samples(likelihood_vector,model,subject):
    
    # model name
    model_name=model.name
    
    #find bad likelihoods
    not_infs= ~np.isinf(likelihood_vector)
    not_nans= ~np.isnan(likelihood_vector)
    valid_parameter_indices=not_infs==not_nans #for retreiving valid parameters
    
    
    # clean liklelihood vector 
    likelihood_vector_noinf=likelihood_vector[~np.isinf(likelihood_vector)] #remove negative infinity likelihoods
    likelihood_vector_cleaned=likelihood_vector_noinf[~np.isnan(likelihood_vector_noinf)] #remove nan likelihoods
    good_samples=len(likelihood_vector_cleaned) #number of usable samples
    
    # log mean likelihood
    model_evidence=np.mean(likelihood_vector_cleaned)
    exec('{}.subject{}.model_evidence_subject={}'.format(model_name,subject,model_evidence))
    
    
    #derive likelihood based weights by dividing by total likelihood (subtract here due to log space)
    weights=np.exp(likelihood_vector_cleaned) 
    
    #get parameters for resampling
    parameters=get_parameters_for_model(model)
    for parameter in parameters:
        exec('global parameter_samples; parameter_samples={}.{}.samples'.format(model_name,parameter))
        parameter_samps=np.reshape(parameter_samples,(model.sample_size,1))
        good_parameters=parameter_samps[valid_parameter_indices]
        mean,ci,sample=compute_samples(good_parameters,weights)
        exec('{}.subject{}.{}.posterior_mean={}'.format(model_name,subject,parameter,mean))
        exec('{}.subject{}.{}.credible_interval={}'.format(model_name,subject,parameter,ci))
        exec('{}.subject{}.{}.samples={}'.format(model_name,subject,parameter,sample))
        exec('{}.subject{}.{}.num_good_samples={}'.format(model_name,subject,parameter,good_samples))


def compute_samples(parameter_samples,weights):
    weights=np.divide(weights,np.sum(weights))
    df=pd.DataFrame()
    df['weights']=weights
    df['parameter_samples']=parameter_samples    
    df.sort_values('parameter_samples') #sort both parameter samples and weights by order of weights
    
    #weighted sum of all parameters by likelihoods
    val=np.sum(df['weights']*df['parameter_samples'])
    cdf=df.weights.cumsum()
    
    #resample from posterior
    samples=[]
    for i in np.arange(0.0001,1,0.001):
        j=next(x[0] for x in enumerate(cdf) if x[1] > i)   
        samples.append(df['parameter_samples'][j])
               
    #find 5% HDI index
    index_5=next(x[0] for x in enumerate(cdf) if x[1] > 0.0499999) 
        
    #find 95% HDI index
    index_95=next(x[0] for x in enumerate(cdf) if x[1] > 0.9499999) 
            
    
    ci_lower=df['parameter_samples'][index_5]
    ci_higher=df['parameter_samples'][index_95]
    ci=[ci_lower,ci_higher]
    
    return val,ci,samples

def fit_hyperparameters(model):
    from scipy.stats import beta,gamma,norm,poisson,uniform,logistic
    parameters=get_parameters_for_model(model)
    number_subjects=model.num_subjects
    model_name=model.name
    for parameter in parameters:
        parameter_full_sample=[]
        exec('global distribution; distribution={}.{}.distribution'.format(model_name,parameter))
        for subject in range(number_subjects):
            exec('parameter_full_sample.append({}.subject{}.{}.samples)'.format(model_name,subject,parameter))
        exec('global hyperparameters; hyperparameters={}.fit(parameter_full_sample)'.format(distribution))
        if distribution=='gamma':
            h1=hyperparameters[0]
            h2=hyperparameters[2]
        else:
            h1=hyperparameters[0]
            h2=hyperparameters[1]
        #record hyperparameters
        exec('{}.{}.hyperparameters={}'.format(model_name,parameter,[h1,h2]))
        #sample from hyperparameters
        exec('{}.{}.samples=sample_parameters("{}",{},{})'.format(model_name,parameter,distribution,[h1,h2],model.sample_size))

def get_total_evidence(model):
    number_subjects=model.num_subjects
    model_name=model.name
    group_model_evidence=0
    for subject in range(number_subjects):
        exec('global subjEvidence; subjEvidence={}.subject{}.model_evidence_subject'.format(model_name,subject))
        group_model_evidence+=subjEvidence
    return group_model_evidence


# Create synthetic datasets for each subject

In [17]:
# create data
import pandas as pd
import numpy as np
from scipy.stats import gamma
from random import shuffle
subjects=2
all_data=[]
for subject in range(subjects):
    exec('data_{}=pd.DataFrame()'.format(subject))
    c=[1,1,1,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,0,0,1]*2
    shuffle(c) #so each simulated subject's choices are different
    exec('data_{}["choice"]=c'.format(subject))
    exec('data_{}["outcome"]=[1,0,1,1,1,0,0,0,0,0,1,1,1,1,1,0,1,1,1,0,0,0,1,1,0,0,0,1,1,1,1,0,0,1]*2'.format(subject))
    exec('all_data.append(data_{})'.format(subject))

# Build models

In [18]:
all_models=[]

#define the number of subjects, parameters, and hyperparameters (i.e., parameters for group prior)

# Moody RL (Eldar & Niv, 2015)
name_1='mood_model'
number_subjects=2
parameter_sample_size=10000
group_parameters_info_mood={'inv_temp':['gamma',[1,1]],'lr_mood':['beta',[1,1]],'mood_bias':['gamma',[1,1]],
                       'lr_val':['beta',[1,1]]} #name of parameters and hyperpriors
likelihood='moodyRL'
build_model(name_1,likelihood,group_parameters_info_mood,number_subjects,parameter_sample_size)
all_models.append(mood_model)

# Standard model-free Q-learner
name_2='RL_model'
group_parameters_info_RL={'inv_temp':['gamma',[1,1]],
                            'lr_val':['beta',[1,1]]} 
likelihood='std_RL'
build_model(name_2,likelihood,group_parameters_info_RL,number_subjects,parameter_sample_size)
all_models.append(RL_model)

# Fit models

In [24]:
#iterate over models
for model in all_models:
    print(model.name)
    improvement=1 #arbitrary start to ensure while loop starts
    
    #keep sampling until no improvement in iBIC
    while improvement>0:
        
        #store old_bic for comparison to new_bic
        old_bic=model.bic

        #generate log likelihood for each subject and compute new samples
        for subject in range(number_subjects):
            data=all_data[subject]
            likelihood=model.lik_func(model,data)
            compute_new_samples(likelihood,model,subject)

        #fit new hyperparameters from full posterior
        fit_hyperparameters(model)
        
        #Compute iBIC
        parameters=get_parameters_for_model(model)
        Nparams = 2*len(parameters)
        Ndatapoints = float(model.num_subjects*len(data['choice'])) #total number of datapoints  
        evidence = get_total_evidence(model) # total evidence
        new_bic = -2.0*float(evidence) + Nparams*np.log(Ndatapoints) # Bayesian Information Criterion
        improvement = old_bic - new_bic # compute improvement of fit
        
        #only retain evidence and BIC if they improve
        if improvement > 0:
            model.model_evidence_group=evidence
            model.bic=new_bic
        
        #read out latest iteration
        print('{}- iBIC old:{}, new: {}\n'.format(model.name, old_bic, model.bic))


mood_model
mood_model- iBIC old:224.8232214648391, new: 224.09638966495658

mood_model- iBIC old:224.09638966495658, new: 222.50267593775905

mood_model- iBIC old:222.50267593775905, new: 222.50267593775905

RL_model
RL_model- iBIC old:202.80299627195805, new: 202.80299627195805



In [28]:
#check subject fits work
mood_model.subject0.model_evidence_subject

-45.19826342005548